In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable


$\textbf{ConvLSTMCell}$定义了$\textbf{ConvLSTM}$的基本单元(Cell)，实现了LSTM的核心逻辑--使用了卷积操作代替了全连接操作

### 传统LSTM的门控机制
$$\mathbf{I}_{t}=\sigma(\mathbf{X}_{t}\mathbf{W}_{xi}+\mathbf{H}_{t-1}\mathbf{W}_{hi}+\mathbf{b}_{i})
\\ 
\mathbf{F}_{t}=\sigma(\mathbf{X}_{t}\mathbf{W}_{xf}+\mathbf{H}_{t-1}\mathbf{W}_{hf}+\mathbf{b}_{f}) \\
\mathbf{O}_{t}=\sigma(\mathbf{X}_{t}\mathbf{W}_{xo}+\mathbf{H}_{t-1}\mathbf{W}_{ho}+\mathbf{b}_{o}) 
\\
\tilde{\mathbf{C}_{t}}=\tanh(\mathbf{X}_{t}\mathbf{W}_{xc}+\mathbf{H}_{t-1}\mathbf{W}_{hc}+\mathbf{b}_{c})(候选细胞状态更新) \\
\mathbf{C}_{t}=\mathbf{F}_{t}\odot \mathbf{C}_{t-1}+\mathbf{I}_{t}\odot \tilde{\mathbf{C}_{t}}(细胞状态更新) \\
\mathbf{H}_{t}=\mathbf{O}_{t}\odot \tanh(\mathbf{C}_{t})
\\

\tag{eq.1} $$

### 修改过的LSTM
## 门的更新（一维卷积）
$$
i_{t} = \sigma(W_{xi} * X_{t} + W_{hi}* H_{t-1} + W_{ci}\odot C_{t-1}+b_{i}) \\
f_{t} = \sigma(W_{xf} * X_{t} + W_{hf}* H_{t-1} + W_{ci}\odot C_{t-1}+b_{f}) \\
C_{t} = f_{t} \odot C_{t-1} + i_{t} \odot \tanh(W_{xc}*X_{t}+W_{hc} \odot H_{t-1} + b_{c}) \\
o_{t} = \sigma(W_{xo}*X_{t} + W_{ho}* H_{t-1}+W_{co}*C_{t}+b_{o}) \\
H_{t} = o_{t}\odot \tanh(C_{t}) \\
\tag{eq.1}
$$
## 主要是权重更新方法---调整bias项
多出了$ W_{ci}$ $W_{cf}$ $W_{co}$---细胞状态的门控权重--用以调节细胞状态在门控机制中的影响
这些权重产生的影响体现在：
$$
c_{i} = \sigma(W_{xi}(x) + W_{hi}(h) + c\odot W_{ci}) (输入门) \\
c_{f} = \sigma(W_{xf}(x) + W_{hf}(h) + c\odot W_{cf}) (遗忘门) \\
c_{o} = \sigma(W_{xo}(x) + W_{ho}(h) + cc\odot W_{co}) (输入门) \\

\tag{eq.2}
$$

以上 $eq.2$ 的基础上，ConvLSTM的前馈神经网络部分（mainly $forward()$） 中还包含了细胞状态更新和隐藏层更新的方程：
$$
c_{c} = \stackrel{\textcircled{1}}{c_f\times c}  + \stackrel{\textcircled{2}}{c_{i} \times \tanh{(W_{xc}+W_{hc})}} \\
c_{h} =  c_{0} \times \tanh{cc}
\tag{eq.3}
$$

$cc$--更新后的细胞状态
$cf\times c$--遗忘门的输出$c_{f}$与上一个时间步的细胞状态 $c$ 相乘，决定保留多少旧信息 \
$\tanh(W_{xc}+W_{hc})$对输入$x$ 和隐状态$h$进行卷积操作，并通过$tanh$激活函数，得到候选细胞状态 \
$ \textcircled{2}$: $c_{i}$与候选细胞状态相乘，决定添加多少新信息 \

$c_{o}\times \tanh(cc)$---更新后的细胞状态$cc$通过$\tanh$函数，得到$\textbf{候选隐状态}$,输出门$c_{o}$ \
与候选隐状态相乘，得到当前时间步的隐状态$c_{h}$

 

$\textbf{只有隐状态会传递到输出层，而记忆元完全属于内部信息。}$

以上 $eq.1$ 中的卷积操作是一维卷积（而不是二维图形数据提取边缘信息的二维卷积矩阵操作）
拿其中 $W_{xi}*X_{t}$表示如下一维卷积过程：
-假设数据$X_{t}$ 是一个长度为5的1D向量： $X_{t}=[1,2,3,4,5]$ 
-卷积核$W_{xi}$是一个长度为3的向量： $W_{xi}=[1,0,-1]$
$$
W_{xi}*X_{t}=[1,0,-1]*[1,2,3,4,5] \\
1\times 1 + 2\times 0+3\times(-1) \\
2\times 1 + 3\times 0 +4\times(-1) \\
3\times 1+4\times 0+5\times (-1)\\
=[-2,-2,-2] \\
\tag{eq.4}
$$
通过以上操作，卷积核 $W_{xi}$帮助模型从输入数据 $X_{t}$提取有用的信息特征（时间序列中的趋势、模式等），并将其整合到隐藏状态中

原始输入数据的shape是 $(N,T,C,D)$，其中：
-N: 样本数

-T: 时间步

-C: 通道数（3个）是

    --fundamental data(spot price, strike price, day to expire, call or put, implied volatility)
    --data of price (previous settle price, settle price change, theory price, theory magin(理论价差))
    --data of greeks (Delta, Gamma, Theta, Vega, Rho)

-D: 每个通道对应的5个特征

为了做对比实验(多通道与单通道的对比)，作者将输入数据重新调整为 $(N,1,T,C \times D)=(N,1,10,15)$

    1:表示只有一个通道

    $C\times D=15$ ：将所有通道的特征拼在一起，即train_data.csv和test_data.csv的column变量都整合在一起

In [2]:
class ConvLSTMCell(nn.Module):
    def __init__(self, input_channels, hidden_channels, kernel_size):
        super(ConvLSTMCell, self).__init__()

        # assert hidden_channels % 2 == 0
        # 输入数据的通道数
        self.input_channels = input_channels
        # 隐藏状态的通道数
        self.hidden_channels = hidden_channels
        # kernel_size：卷积核的大小
        self.kernel_size = kernel_size
        self.num_features = 4

        self.padding = int((kernel_size - 1) / 2)
        # W_{xi}}----输入门中输入X_{t}的权重
        self.Wxi = nn.Conv1d(self.input_channels, self.hidden_channels, self.kernel_size, 1, self.padding, bias=True)
        # W_{hi}----输入门中 隐状态 H_{t-1} 的权重
        self.Whi = nn.Conv1d(self.hidden_channels, self.hidden_channels, self.kernel_size, 1, self.padding, bias=False)
        # W_{xf}---遗忘门中 输入X_{t}的权重
        self.Wxf = nn.Conv1d(self.input_channels, self.hidden_channels, self.kernel_size, 1, self.padding, bias=True)
        # W_{hf}---遗忘门中 隐状态 H_{t-1}的权重
        self.Whf = nn.Conv1d(self.hidden_channels, self.hidden_channels, self.kernel_size, 1, self.padding, bias=False)
        # W_{xc}---输入 X_{t}的细胞状态
        self.Wxc = nn.Conv1d(self.input_channels, self.hidden_channels, self.kernel_size, 1, self.padding, bias=True)
        # W_{hc}--- 隐状态 H_{t-1}的细胞状态
        self.Whc = nn.Conv1d(self.hidden_channels, self.hidden_channels, self.kernel_size, 1, self.padding, bias=False)
        # 输出门的权重
        self.Wxo = nn.Conv1d(self.input_channels, self.hidden_channels, self.kernel_size, 1, self.padding, bias=True)
        
        self.Who = nn.Conv1d(self.hidden_channels, self.hidden_channels, self.kernel_size, 1, self.padding, bias=False)

        self.Wci = None
        self.Wcf = None
        self.Wco = None
        
    '''
    forward(self, x, h, c)用于计算当前时间步的隐状态(hidden state)和细胞态(cell state)
    input: 当前时间步的输入数据--x; 上一个时间步的隐状态--h;细胞状态--c
    output: 当前时间步的隐状态 ch 和细胞状态 cc
    '''
    def forward(self, x, h, c):
        ci = torch.sigmoid(self.Wxi(x) + self.Whi(h) + c * self.Wci)
        cf = torch.sigmoid(self.Wxf(x) + self.Whf(h) + c * self.Wcf)
        cc = cf * c + ci * torch.tanh(self.Wxc(x) + self.Whc(h))
        co = torch.sigmoid(self.Wxo(x) + self.Who(h) + cc * self.Wco)
        ch = co * torch.tanh(cc)
        return ch, cc
    # 为LSTM提供初始的隐状态和细胞状态
    # batch_size--每次更新模型参数时所需的样本数量
    # dataset一次训练不完，需要分多次小批次分批训练
    # hidden--隐状态的通道数（即hidden_channels）
    # dim--输入数据的维度
    # nn.Parameter() 将张量包装为可训练的参数
    # .cuda() 将参数移动到CUDA上(如果GPU可用)
    def init_hidden(self, batch_size, hidden, dim):
        if self.Wci is None:
            self.Wci = nn.Parameter(torch.zeros(1, hidden, dim)).cuda()
            self.Wcf = nn.Parameter(torch.zeros(1, hidden, dim)).cuda()
            self.Wco = nn.Parameter(torch.zeros(1, hidden, dim)).cuda()
        else:
            # 如果self.Wci已经初始化 检查输入数据的维度 dim是否与self.Wci第三维度匹配
            assert dim == self.Wci.size()[2], 'Input Dim Mismatched!'
        return (Variable(torch.zeros(batch_size, hidden, dim)).cuda(),
                Variable(torch.zeros(batch_size, hidden, dim)).cuda())
        # 返回的两个对象-h和c--初始化的隐状态+细胞状态

In [3]:
class ConvLSTM(nn.Module):
    # input_channels corresponds to the first input feature map
    # hidden state is a list of succeeding lstm layers.
    def __init__(self, input_channels, hidden_channels, kernel_size, in_dim, out_dim, step=10):
        super(ConvLSTM, self).__init__()
        # 将输入通道数和隐藏通道数合并为同一个列表，表示为每一层的输入通道数
        self.input_channels = [input_channels] + hidden_channels
        self.hidden_channels = hidden_channels
        self.kernel_size = kernel_size
        # LSTM的层数 等于hidden_channels的长度  
        self.num_layers = len(hidden_channels)
        # 时间步数，表示输入序列的长度
        self.step = step
        self._all_layers = []
        # 定义一个全连接层，用于将LSTM的输出映射到目标维度--最后一个得到optionprice的步骤
        self.linear = nn.Linear(in_features=in_dim, out_features=out_dim)

        for i in range(self.num_layers):
            # 每一个层生成一个唯一的名称， 例如cell0, cell1
            name = 'cell{}'.format(i)
            cell = ConvLSTMCell(self.input_channels[i], self.hidden_channels[i], self.kernel_size)
            ## 将ConvLSTMCell实例动态添加到 ConvLSTM类中，使其成为类的属性
            setattr(self, name, cell)
            self._all_layers.append(cell)
# N--观察数
# C--通道数channels
# D--变量(Delta Gamma Theta Vega Rho)
    def forward(self, input):
        # input:(N, C, T, D_in)
        # N--batch size---一次处理的数据样本数量
        # C--channels--每个时间步的输入特征数
        # T--time steps--时间序列的长度
        # D_in --- input dimension---每个时间步的特征向量的维度
        # internel_state-用于存储每一层的LSTM单元的隐藏状态(h,c) 其中 h--hidden state, c--cell state
        internal_state = []
        '''
        假设 
        N=2----2个样本
        C=3----3个通道
        T=3----3个时间步
        D_in=2-每个时间步的特征向量有2维
        input = torch.tensor([
    # 样本 1
    [
        # 通道 1
        [[1.1, 1.2], [2.1, 2.2], [3.1, 3.2]],
        # 通道 2
        [[1.3, 1.4], [2.3, 2.4], [3.3, 3.4]],
        # 通道 3
        [[1.5, 1.6], [2.5, 2.6], [3.5, 3.6]]
    ],
    # 样本 2
    [
        # 通道 1
        [[4.1, 4.2], [5.1, 5.2], [6.1, 6.2]],
        # 通道 2
        [[4.3, 4.4], [5.3, 5.4], [6.3, 6.4]],
        # 通道 3
        [[4.5, 4.6], [5.5, 5.6], [6.5, 6.6]]
    ]
])
        
        '''
        # self.step = 10 是是个样本
        for step in range(self.step):
            x = input[:, :, step, :]
            for i in range(self.num_layers):
                # all cells are initialized in the first step
                name = 'cell{}'.format(i)
                if step == 0:
                    bsize, _, dim = x.size()
                    # 获取第 i 层的LSTM单元
                    # 调用LSTM单元初始化方法，返回初始化的隐藏状态（h,c）
                    (h, c) = getattr(self, name).init_hidden(batch_size=bsize, hidden=self.hidden_channels[i],
                                                             dim=dim)
                    internal_state.append((h, c))

                # do forward
                (h, c) = internal_state[i]
                x, new_c = getattr(self, name)(x, h, c)
                internal_state[i] = (x, new_c)
            outputs = x
            # outputs:(N, hidden_channels[-1], D_in)
            outputs = self.linear(outputs)
            # outputs:(N, hidden_channels[-1], D_out)

        return outputs.squeeze()

In [4]:
# test
if __name__ == '__main__':
    input = torch.normal(0, 1, size=(64, 3, 10, 5)).cuda()
    model = ConvLSTM(input_channels=3, hidden_channels=[3, 1], kernel_size=3, in_dim=5, out_dim=1, step=10).cuda()
    output = model.forward(input)
    print(output.shape)

torch.Size([64])


假设

`input = torch.normal(0, 1, size=(64, 3, 10, 5)).cuda()` 

+ N=64--batch size
+ C=3--channels
+ T=10--time steps(same option in past 10 days)
+ D_in=5--5 dimensions in every timestep(fundamental data, data of price, data of greeks) 

model initalization:
`model = ConvLSTM(input_channels=3, hidden_channels=[3,1], kernel_size=3, in_dim=5, out_dim=1, step=10).cuda()`
- `hidden_channels=[3,1]`: two layers of LSTM units, hidden channels of layer 1 is 3, layer 2 is 1

### Running process
1. model initializaiton will initialize two layers of LSTM units:
- 1st layer: `ConvLSTMCell(input_channels=3, hidden_channels=3, kernel_size=3)`
- 2nd layer: `ConvLSTMCell(input_channels=3, hidden_channels=1, kernel_size=3)`

2. forward
output = model.forward(input)
dimention of `input` is `(64, 3, 10, 5)`

    2.1 loops by time step
    `x=input[:, :, step ,:]`
    - dimension of x is `(64, 3, 5)`--represents features of  all samples and channels of current time step
    2.2 loops by layer
    for every layer i (0 to 1):
    - initialize state  (h,c) in case of `step==0`:
    `(h,c)=getattr(self, name).init_hidden(batch_size=64, hidden=self.hidden_channels[i], dim=5)`
        - 1st layer : dimension of `h` and `c` is (64, 3, 5)
        - 2nd layer : dimension of `h` and `c` is (64, 1, 5) 
    - do forward of LSTM
    `x, new_c=getattr(self, name)(x, h, c) `
    - update hidden sate:
    `internal_state[i]=(x, new_c)`

    - process output
    at the last time `step=9` , `x` is the output of layer 2, dimension is (64, 1, 5)
    we can map the output to the target dimension through a linear layer:
    `outputs = self.linear(outputs)` dimension of `outputs` is  `(64, 1, 1)`
    - return results
    `return outputs.squeeze()`
    dimension of outputs is (64,)
